# IBM Applied Data Science Capstone Course by Coursera

Opening a New Shopping Mall in New Delhi, India


-Build a dataframe of neighborhoods in New Delhi, India by scraping the data from Wikipedia page  
-Get the geographical coordinates of the neighborhoods  
-Obtain the venue data for the neighborhoods from Foursquare API  
-Explore and cluster the neighborhoods  
-Select the best cluster to open a new shopping mall  

In [1]:
# Web Scraping
!pip install BeautifulSoup4
!pip install requests
!pip install lxml
!pip install geopandas
!pip install geopy
!pip install geocoder
#importing libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests
import geocoder # library to handle requests
from bs4 import BeautifulSoup 
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 112kB 25.1MB/s eta 0:00:01
     |████████████████████████████████| 5.8MB 6.6MB/s eta 0:00:01
     |████████████████████████████████| 931kB 6.7MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 9.1MB/s eta 0:00:01     |███████▊                        | 440kB 9.1MB/s eta 0:00:01
     |████████████████████████████████| 14.7MB 7.0MB/s eta 0:00:011       | 9.8MB 30.2MB/s eta 0:00:0101
     |████████████████████████████████| 10.9MB 16.9MB/s eta 0:00:01███████▏                   | 4.2MB 16.9MB/s eta 0:00:01
     |████████████████████████████████| 92kB 7.9MB/s  eta 0:00:01
  Found existing installation: pyproj 1.9.6
    Uninstalling pyproj-1.9.6:
      Successfully uninstalled pyproj-1.9.6
     |████████████████████████████████| 122kB 6.2MB/s eta 0:00:01
     |████████████████████████████████| 102kB 6.1MB/s ta 0:00:011
     |████████████████████████████████| 829kB 9.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8

# Scrap data from Wikipedia page into a DataFrame

In [2]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Delhi").text

In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
# create a list to store neighborhood data
neighborhoodList = []

In [5]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
neighborhoodList.remove('Neighbourhoods of Delhi')

In [6]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})
kl_df.head()

,Neighborhood
0,Ashok Nagar (Delhi)
1,Ashok Vihar
2,Ashram Chowk
3,Babarpur
4,"Badarpur, Delhi"


In [7]:
kl_df.shape

(141, 1)

## Getting geographical cordinates

In [8]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, New Delhi, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [10]:
coords

[[28.692230000000052, 77.30124000000006],
 [28.69037000000003, 77.17609000000004],
 [28.710597501792023, 77.32696517369723],
 [28.50738000000007, 77.30346000000003],
 [28.50738000000007, 77.30346000000003],
 [28.652234222889238, 77.12939224396462],
 [28.79767000000004, 77.04522000000003],
 [28.549540000000036, 77.18167000000005],
 [28.699880000000064, 77.25906000000003],
 [28.595060000000046, 77.18573000000004],
 [28.656270000000063, 77.23232000000007],
 [28.538400000000024, 77.24832000000004],
 [28.67671000000007, 77.21767000000006],
 [28.633940000000052, 77.21968000000004],
 [28.60761000000008, 77.08714000000003],
 [28.65457890544559, 77.23339989939495],
 [28.62832000000003, 77.24727000000007],
 [28.605920000000026, 77.08529000000004],
 [28.560590000000047, 77.24678000000006],
 [28.57298000000003, 77.23357000000004],
 [28.591510000000028, 77.12945000000008],
 [28.699110000000076, 77.19105000000008],
 [28.594857177133914, 77.16729160908383],
 [28.684700000000078, 77.32774000000006],
 

In [11]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [12]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [13]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(141, 3)


,Neighborhood,Latitude,Longitude
0,Ashok Nagar (Delhi),28.692230,77.301240
1,Ashok Vihar,28.690370,77.176090
2,Ashram Chowk,28.710598,77.326965
3,Babarpur,28.507380,77.303460
4,"Badarpur, Delhi",28.507380,77.303460
5,Bali Nagar,28.652234,77.129392
6,Bawana,28.797670,77.045220
7,Ber Sarai,28.549540,77.181670
8,Bhajanpura,28.699880,77.259060
9,Chanakyapuri,28.595060,77.185730


In [14]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

## Create a map of New Delhi with neighborhoods superimposed on top

In [15]:
# get the coordinates of New Delhi
address = 'New Delhi, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New Delhi, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New Delhi, India 28.6141793, 77.2022662.


In [16]:
# create map of New Delhi using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [17]:
# save the map as HTML file
map_kl.save('map_kl.html')

## Use the Foursquare API to explore the neighborhoods

In [18]:
# define Foursquare Credentials and Version
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20200202' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


 ### Top 100 venues that are within a radius of 2000 meters

In [19]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

KeyError: 'groups'

In [20]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

ValueError: Length mismatch: Expected axis has 0 elements, new values have 7 elements

### How many venues were returned for each neighorhood

In [21]:
venues_df.groupby(["Neighborhood"]).count()

KeyError: 'Neighborhood'

### How many unique categories can be curated from all the returned venues

In [22]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

KeyError: 'VenueCategory'

In [23]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

KeyError: 'VenueCategory'

In [24]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

KeyError: 'VenueCategory'

## Analyze Neighborhood

In [25]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

KeyError: "None of [Index(['VenueCategory'], dtype='object')] are in the [columns]"

### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [26]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

NameError: name 'kl_onehot' is not defined

In [ ]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

In [ ]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [ ]:
kl_mall.head()

# Cluster Neighborhoods

 Run k-means to cluster the neighborhoods in New Delhi into 3 clusters.

In [ ]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [27]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

NameError: name 'kl_mall' is not defined

In [ ]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

In [ ]:
# merge dataFrames to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

In [ ]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

### Finally, let's visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

# Examine Clusters

 Cluster 0 

In [28]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0].shape

NameError: name 'kl_merged' is not defined

In [ ]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]


Cluster 1

In [ ]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1].shape

In [ ]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

Cluster 2

In [ ]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2].shape

In [ ]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

### OBSERVATIONS:

Cluster 0 has highest number(116) of shopping mall. On the other hand, cluster 1 has very less shopping malls as compared to other two clusters. This represents a great opportunity and high potential areas to open new shopping malls in cluster 1 as there is very little competition. Meanwhile, shopping malls in cluster 0 are likely suffering from intense competition due to oversupply and high concentration of shopping malls.

**Therefore, the outcomes of this project recommend property developers to open new shopping malls in neighborhoods in cluster 1.**

Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 2 with moderate competition.
**Lastly, property developers are advised to avoid neighborhoods in cluster 0 which already have high concentration of shopping malls.**